In [1]:
import json
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
with open('cocolvis_cp/val2017/hannotation.pickle', 'rb') as f:
    data = pickle.load(f)

In [3]:
df = pd.DataFrame(columns=['img_id', 'obj_id', 'syn_gt', 'syn_gt_void', 'syn_annotations', 'prior', 'corr_id', 'gesture', 'mode'])

In [4]:
gts = os.listdir('cocolvis_cp/val2017/gts_og')
im_ids = [i[:-4] for i in gts]
im_ids[:10]

['000000520910',
 '000000013201',
 '000000445846',
 '000000368752',
 '000000064868',
 '000000135410',
 '000000556158',
 '000000437110',
 '000000376625',
 '000000241326']

In [38]:
p_annots = sorted(os.listdir('cocolvis_cp/val2017/new_fl_syn_annotations'))
p_gts = sorted(os.listdir('cocolvis_cp/val2017/new_fl_syn_gts'))
priors = sorted(os.listdir('cocolvis_cp/val2017/new_fl_syn_init_masks'))

p_annots[0]

'000000000139_00001_000_boundary_add.png'

In [41]:
pa_dir = 'cocolvis_cp/val2017/new_fl_syn_annotations'
pg_dir = 'cocolvis_cp/val2017/new_fl_syn_gts'

In [39]:
tmp = im_ids[0]
tmpp = [i for i in priors if tmp in i]
tmpp

['000000520910_00001_000_boundary_sub.png',
 '000000520910_00001_000_box_sub.png',
 '000000520910_00001_000_circle_sub.png',
 '000000520910_00001_000_click_sub.png',
 '000000520910_00001_000_stroke_sub.png',
 '000000520910_00001_001_boundary_add.png',
 '000000520910_00001_001_box_add.png',
 '000000520910_00001_001_circle_add.png',
 '000000520910_00001_001_click_add.png',
 '000000520910_00001_001_stroke_add.png',
 '000000520910_00001_002_boundary_sub.png',
 '000000520910_00001_002_box_sub.png',
 '000000520910_00001_002_circle_add.png',
 '000000520910_00001_002_click_sub.png',
 '000000520910_00001_002_stroke_sub.png',
 '000000520910_00002_000_boundary_sub.png',
 '000000520910_00002_000_box_sub.png',
 '000000520910_00002_000_circle_sub.png',
 '000000520910_00002_000_click_sub.png',
 '000000520910_00002_000_stroke_sub.png',
 '000000520910_00006_000_boundary_add.png',
 '000000520910_00006_000_box_add.png',
 '000000520910_00006_000_circle_add.png',
 '000000520910_00006_000_click_add.png',
 '

In [6]:
nop_annots = sorted(os.listdir('cocolvis_cp/val2017/syn_annotations'))
nop_gts = sorted(os.listdir('cocolvis_cp/val2017/syn_gts'))
assert len(nop_annots) == len(nop_gts)
nopa_dir = 'cocolvis_cp/val2017/syn_annotations'
nopg_dir = 'cocolvis_cp/val2017/syn_gts'

In [7]:
def get_corr_id(fpath):
    if len(fpath.split('_')) == 5:
        return int(fpath.split('_')[2])
    return -1

def get_obj_id(fpath):
    return int(fpath.split('_')[1])

def has_prior(fpath):
    prior = fpath.split('_')[-1][:-4]
    if prior == 'noprior':
        return False
    return True

def get_mode(fpath):
    mode = fpath.split('_')[-1][:-4]
    if mode == 'sub':
        return 0
    return 1

get_corr_id(p_annots[0]), get_corr_id(nop_annots[0]), has_prior(p_annots[0]), has_prior(nop_annots[0])

(0, -1, True, False)

In [15]:
def create_blank_row():
    row = {
            'img_id': None,
            'size': None,
            'obj_id': None,
            'syn_gt': None,
            'syn_gt_void': None,
            'syn_annotations': None,
            'has_prior': None,
            'prior': None,
            'corr_id': None,
            'gesture': None,
            'mode': None,
        }
    return row

In [34]:
def process_nop(id_, nop_annots, nop_gts, gesture_dict, nopa_dir, nopg_dir):
    tmp_annots_names = sorted([i for i in nop_annots if id_ in i])
    tmp_gts_names = sorted([i for i in nop_gts if id_ in i])
    
    row = create_blank_row()
    row['img_id'] = id_
    _obj_ids = list(map(lambda x: get_obj_id(x), tmp_annots_names))
    row['obj_id'] = _obj_ids
    
    tmp_annots = [cv2.imread(f"{nopa_dir}/{i}")[:,:,0] for i in tmp_annots_names]
    tmp_gts = [cv2.imread(f"{nopg_dir}/{i}")[:,:,0] for i in tmp_gts_names]
    row['size'] = tmp_annots[0].shape
    _valids = [list(np.where(i == 1)) for i in tmp_gts]
    _voids = [list(np.where(i == 255)) for i in tmp_gts]
    row['syn_gt'] = _valids
    row['syn_gt_void'] = _voids
    
    _annots = [list(np.where(i == 255)) for i in tmp_annots]
    row['syn_annotations'] = _annots
    
    _gests = [i.split('_')[-2] for i in tmp_annots_names]
    _gests = [gesture_dict[i] for i in _gests]
    _modes = [i.split('_')[-1][:-4] for i in tmp_annots_names]
    row['gesture'] = _gests
    row['mode'] = _modes
    _corr_ids = list(map(lambda x: get_corr_id(x), tmp_annots_names))
    row['corr_id'] = _corr_ids 
    _priors = list(map(lambda x: has_prior(x), tmp_annots_names))
    row['has_prior'] = _priors
    row['prior'] = []#[[-1] for i in _priors]
    
    _modes = list(map(lambda x: get_mode(x), tmp_annots_names))
    row['mode'] = _modes
    return row

In [66]:
def process_prior(id_, p_annots, p_gts, priors, gesture_dict, pa_dir, pg_dir):
    tmp_annots_names = sorted([i for i in p_annots if id_ in i])
    tmp_gts_names = sorted([i for i in p_gts if id_ in i])
    print(len(tmp_annots_names), len(tmp_gts_names))
    for i in range(len(tmp_annots_names)):
        print(tmp_annots_names[i] == tmp_gts_names[i],tmp_annots_names[i],tmp_gts_names[i])
    prior = [i for i in priors if id_ in i][0]
    prior = cv2.imread(prior, cv2.IMREAD_GRAYSCALE)
    
    
    row = create_blank_row()
    row['img_id'] = id_
    _obj_ids = list(map(lambda x: get_obj_id(x), tmp_annots_names))
    row['obj_id'] = _obj_ids
    
    tmp_annots = [cv2.imread(f"{pa_dir}/{i}")[:,:,0] for i in tmp_annots_names]
    tmp_gts = [cv2.imread(f"{pg_dir}/{i}")[:,:,0] for i in tmp_gts_names]
    row['size'] = tmp_annots[0].shape
    _valids = [list(np.where(i == 1)) for i in tmp_gts]
    _voids = [list(np.where(i == 255)) for i in tmp_gts]
    row['syn_gt'] = _valids
    row['syn_gt_void'] = _voids
    
    _annots = [list(np.where(i == 255)) for i in tmp_annots]
    row['syn_annotations'] = _annots
    
    _gests = [i.split('_')[-2] for i in tmp_annots_names]
    _gests = [gesture_dict[i] for i in _gests]
    _modes = [i.split('_')[-1][:-4] for i in tmp_annots_names]
    row['gesture'] = _gests
    row['mode'] = _modes
    _corr_ids = list(map(lambda x: get_corr_id(x), tmp_annots_names))
    row['corr_id'] = _corr_ids 
    _priors = list(map(lambda x: has_prior(x), tmp_annots_names))
    row['has_prior'] = _priors
    row['prior'] = [prior] #[[-1] for i in _priors]
    
    _modes = list(map(lambda x: get_mode(x), tmp_annots_names))
    row['mode'] = _modes
    return row

In [67]:
tmp = process_prior(im_ids[0], p_annots, p_gts, priors, gesture_dict, pa_dir, pg_dir)

70 100
True 000000520910_00001_000_boundary_sub.png 000000520910_00001_000_boundary_sub.png
True 000000520910_00001_000_box_sub.png 000000520910_00001_000_box_sub.png
True 000000520910_00001_000_circle_sub.png 000000520910_00001_000_circle_sub.png
True 000000520910_00001_000_click_sub.png 000000520910_00001_000_click_sub.png
True 000000520910_00001_000_stroke_sub.png 000000520910_00001_000_stroke_sub.png
True 000000520910_00001_001_boundary_add.png 000000520910_00001_001_boundary_add.png
True 000000520910_00001_001_box_add.png 000000520910_00001_001_box_add.png
True 000000520910_00001_001_circle_add.png 000000520910_00001_001_circle_add.png
True 000000520910_00001_001_click_add.png 000000520910_00001_001_click_add.png
True 000000520910_00001_001_stroke_add.png 000000520910_00001_001_stroke_add.png
True 000000520910_00001_002_boundary_sub.png 000000520910_00001_002_boundary_sub.png
True 000000520910_00001_002_box_sub.png 000000520910_00001_002_box_sub.png
True 000000520910_00001_002_cir

In [35]:
process_nop(im_ids[0], nop_annots, nop_gts, gesture_dict)['size']

(640, 427)

In [36]:
(np.arange(9).reshape((3,3)) == -1).any(), (np.array([-1]) == -1).any()

(False, True)

In [37]:
df = pd.DataFrame(columns=['img_id', 'size','obj_id', 'syn_gt', 'syn_gt_void', 'syn_annotations', 'has_prior',
                           'prior', 'corr_id', 'gesture', 'mode'])
for id_ in im_ids[:5]:
    row = process_nop(id_, nop_annots, nop_gts, gesture_dict)
    df = df.append(row, ignore_index = True)
df.head()

,img_id,size,obj_id,syn_gt,syn_gt_void,syn_annotations,has_prior,prior,corr_id,gesture,mode
0,000000520910,"(640, 427)","[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, ...","[[[162, 163, 163, 163, 163, 163, 163, 163, 163...","[[[167, 168, 169], [213, 214, 215]], [[], []],...","[[[157, 158, 158, 158, 158, 158, 158, 158, 159...","[False, False, False, False, False, False, Fal...","[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,000000013201,"(640, 427)","[1, 1, 1, 1, 1, 3, 3, 3, 3, 3]","[[[87, 88, 88, 88, 89, 89, 89, 89, 89, 90, 90,...","[[[288, 290, 437, 437, 438, 438, 438, 438, 438...","[[[83, 83, 83, 83, 83, 83, 83, 84, 84, 84, 84,...","[False, False, False, False, False, False, Fal...","[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,000000445846,"(427, 640)","[1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, ...","[[[267, 267, 267, 267, 267, 267, 267, 267, 267...","[[[], []], [[], []], [[], []], [[], []], [[], ...","[[[243, 243, 243, 243, 243, 244, 244, 244, 244...","[False, False, False, False, False, False, Fal...","[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,000000368752,"(640, 480)","[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, ...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[], []], [[], []], [[], []], [[], []], [[], ...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[False, False, False, False, False, False, Fal...","[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,000000064868,"(375, 500)","[1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, ...","[[[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9...","[[[117, 121, 126, 131, 134, 135, 136, 138, 139...","[[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5...","[False, False, False, False, False, False, Fal...","[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [10]:
# gesture_dict = {
#     'click': 0,
#     'stroke': 1,
#     'circle': 2,
#     'boundary': 3,
#     'box': 4
# }
# df = pd.DataFrame(columns=['img_id', 'size','obj_id', 'syn_gt', 'syn_gt_void', 'syn_annotations', 'prior', 'corr_id', 'gesture', 'mode'])
# for id_ in im_ids[:10]:
#     tmp_annots_names = sorted([i for i in nop_annots if id_ in i])
#     tmp_gts_names = sorted([i for i in nop_gts if id_ in i])
    
#     row = create_blank_row()
#     row['img_id'] = id_
#     _obj_ids = list(map(lambda x: get_obj_id(x), tmp_annots_names))
#     row['obj_id'] = _obj_ids
    
#     tmp_annots = [cv2.imread(f"{nopa_dir}/{i}")[:,:,0] for i in tmp_annots_names]
#     tmp_gts = [cv2.imread(f"{nopg_dir}/{i}")[:,:,0] for i in tmp_gts_names]
#     _valids = [list(np.where(i == 1)) for i in tmp_gts]
#     _voids = [list(np.where(i == 255)) for i in tmp_gts]
#     row['syn_gt'] = _valids
#     row['syn_gt_void'] = _voids
    
#     _annots = [list(np.where(i == 255)) for i in tmp_annots]
#     row['syn_annotations'] = _annots
    
#     _gests = [i.split('_')[-2] for i in tmp_annots_names]
#     _gests = [gesture_dict[i] for i in _gests]
#     _modes = [i.split('_')[-1][:-4] for i in tmp_annots_names]
#     row['gesture'] = _gests
#     row['mode'] = _modes
#     _corr_ids = list(map(lambda x: get_corr_id(x), tmp_annots_names))
#     row['corr_id'] = _corr_ids 
#     _priors = list(map(lambda x: has_prior(x), tmp_annots_names))
#     row['prior'] = _priors
    
#     _modes = list(map(lambda x: get_mode(x), tmp_annots_names))
#     row['mode'] = _modes
    
#     df = df.append(row, ignore_index = True)
# df.head()

,img_id,obj_id,syn_gt,syn_gt_void,syn_annotations,prior,corr_id,gesture,mode
0,000000520910,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, ...","[[[162, 163, 163, 163, 163, 163, 163, 163, 163...","[[[167, 168, 169], [213, 214, 215]], [[], []],...","[[[157, 158, 158, 158, 158, 158, 158, 158, 159...","[False, False, False, False, False, False, Fal...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,000000013201,"[1, 1, 1, 1, 1, 3, 3, 3, 3, 3]","[[[87, 88, 88, 88, 89, 89, 89, 89, 89, 90, 90,...","[[[288, 290, 437, 437, 438, 438, 438, 438, 438...","[[[83, 83, 83, 83, 83, 83, 83, 84, 84, 84, 84,...","[False, False, False, False, False, False, Fal...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,000000445846,"[1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, ...","[[[267, 267, 267, 267, 267, 267, 267, 267, 267...","[[[], []], [[], []], [[], []], [[], []], [[], ...","[[[243, 243, 243, 243, 243, 244, 244, 244, 244...","[False, False, False, False, False, False, Fal...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,000000368752,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, ...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[], []], [[], []], [[], []], [[], []], [[], ...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[False, False, False, False, False, False, Fal...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,000000064868,"[1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, ...","[[[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9...","[[[117, 121, 126, 131, 134, 135, 136, 138, 139...","[[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5...","[False, False, False, False, False, False, Fal...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[3, 4, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
nop_annots[0]

In [53]:
!pwd

/pl/active/pl_joshmyersdean/datasets/dig
